In [1]:
import pandas as pd
import datetime
import numpy as np
from funciones import limpiar_caracteres_especiales
from funciones import categorizacion_fondos_wos, filtrar_normalizacion_funding_wos, normalizar_casa_publicacion
from area_ocde import wos_to_ocde

In [9]:
#ciencias naturales, ingenieria y tecnologia, ciencias medicas y de la salud, ciencias agricolas, ciencias sociales, humanidades
def vector_ocde(vector, area):
    if area=='ciencias naturales':
        vector[0] = 'si'
    elif area=='ingenieria y tecnologia':
        vector[1] = 'si'
    elif area=='ciencias medicas y de la salud':
        vector[2] = 'si'
    elif area=='ciencias agricolas':
        vector[3] = 'si'
    elif area=='ciencias sociales':
        vector[4] = 'si'
    elif area=='ciencias humanidades':
        vector[5] = 'si'
    return vector

### Trabajos WOS

In [10]:
tmp_xls = []
lista_areas = []

for n in [1, 2]:
    xls_base = pd.read_excel(f"raw-data/wos/chile-2010-2023({n}).xls")
    for i,row in enumerate( xls_base.to_numpy().tolist() ):
        authors_short = [ limpiar_caracteres_especiales( n.lower().strip() ) for n in row[0].split(';') ]
        authors_full = [ limpiar_caracteres_especiales( n.lower().strip() ) for n in row[1].split(';') ]
    
        source_title = limpiar_caracteres_especiales( row[2].lower().strip() )
        funding = row[15]
        funding = funding.split(';')
        funding = [ categorizacion_fondos_wos(f) for f in funding ]
        anid, inach, extranjero, corfo, otro, uni = filtrar_normalizacion_funding_wos( funding )

        year = row[30]
        doi = row[31]
        address = row[9]

        num_cites = row[20]

        affiliation = limpiar_caracteres_especiales( row[9].lower().strip() )
        aff_paper = 'no'
        if 'chile' in affiliation:
            aff_paper = 'si'

        n_aff_chile = 0
        n_aff_inter = 0
        for person_full, person_short in zip(authors_full, authors_short):
            if any([ person_full in aff or person_short in aff for aff in affiliation.split(';') if 'chile' in aff]): 
                n_aff_chile += 1
            else:
                n_aff_inter += 1
            
        publish_house = normalizar_casa_publicacion( row[23] ) 

        theme = limpiar_caracteres_especiales( row[32].lower()).split(";")
        #ciencias naturales, ingenieria y tecnologia, ciencias medicas y de la salud, ciencias agricolas, ciencias sociales, humanidades
        vec_ocde = ['no', 'no', 'no', 'no', 'no', 'no']
        if type(theme) == str:
            tema = wos_to_ocde[theme]
            vec_ocde = vector_ocde( vec_ocde, tema )
        else:
            for t in theme:
                tema = wos_to_ocde[t.strip()]
                vec_ocde = vector_ocde( vec_ocde, tema )
        
        tmp_xls.append([source_title, doi, year, anid, inach, extranjero, corfo, 
            uni, otro, aff_paper, n_aff_chile, n_aff_inter, n_aff_chile+n_aff_inter, publish_house, num_cites,
            vec_ocde[0], vec_ocde[1], vec_ocde[2], vec_ocde[3], vec_ocde[4], vec_ocde[5]])

In [11]:
fecha = datetime.datetime.now()

full_dataframe = pd.DataFrame(np.array(tmp_xls), columns=['titulo', 'doi', 'anio', 'fin-anid', 'fin-inach', 'fin-extranjero', 
        'fin-corfo', 'fin-uni', 'fin-otro', 'aff-chile', 'n-chilenos', 'n-extranjeros', 'n-personas', 'publisher', 'cites',
        'a1', 'a2', 'a3', 'a4', 'a5', 'a6'])
full_dataframe.to_excel(f"wos/WOS-todos-papers.xlsx")